In [ ]:
from tensorflow.keras.optimizers import Adam , RMSprop 
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint , LearningRateScheduler
from tensorflow.keras.models import Model
import keras
import cv2
import numpy as np
from Model import get_model

In [ ]:
NEW_DATA_DIR = './data'
TRAIN_DIR = NEW_DATA_DIR + '/Train'
TEST_DIR = NEW_DATA_DIR + '/Test'
VALID_DIR = NEW_DATA_DIR + '/Valid'
test_size = 0.10


def blur(img):
    rand_int = np.random.randint(11)
    if rand_int <= 1:
        blurred = (cv2.blur(img,(5,5)))
        return blurred
    if rand_int <= 3:
        kernel = np.ones((6, 6), np.float32)/20
        filtered = cv2.filter2D(img, -1, kernel)
        return filtered
    return img

#image_size = (136, 204)
image_size = (299, 299)
batch_size = 10

# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,
    vertical_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    fill_mode='nearest',
    shear_range=0.1,
    rescale=1/255,
    brightness_range=[0.5, 1.5],
    preprocessing_function= blur)
validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                TRAIN_DIR,
                target_size=(image_size[0], image_size[1]),
                batch_size=batch_size,
                class_mode='categorical',
                shuffle=True)

# Flow validation images in batches of 20 using test_datagen generator
test_generator = validation_datagen.flow_from_directory(
                TEST_DIR, # Source directory for the validation images
                target_size=(image_size[0], image_size[1]),
                batch_size=batch_size,
                class_mode='categorical',
                shuffle=True)

In [ ]:
nets = ['MobileNet','VGG16' 'MobileNet' 'DenseNet201', 'Inception','ResNet']
num_epochs = 85
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=test_generator.n//test_generator.batch_size

histories = []
for network in nets:
    print('---------------------- TRAIN %s ----------------------'%network)
    reduce_lr =  ReduceLROnPlateau(monitor = "val_loss", factor = 0.5, patience = 10,verbose = 0, mode = "auto", epsilon = 1e-04, cooldown = 0,min_lr = 1e-5)
    es = EarlyStopping(monitor = "val_loss" , verbose = 1 , mode = 'min' , patience = 50 )
    mc = ModelCheckpoint('checkpoints/%s/best_model_loss.h5'%network, monitor = 'loss' , mode = 'min', verbose = 1 , save_best_only = True)
    mc_val = ModelCheckpoint('checkpoints/%s/best_model_val_loss.h5'%network, monitor = 'val_loss' , mode = 'min', verbose = 1 , save_best_only = True)
    model = get_model(network)
    model.load_weights('checkpoints/%s/best_model_val_loss.h5'%network)
    history = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_generator,
                    validation_steps=STEP_SIZE_VALID,
                    callbacks=[reduce_lr,es,mc,mc_val],
                    epochs=num_epochs)
    histories.append(history)